In [1]:
from pprint import pprint
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [1]:
from blogging_app.models import *


In [14]:
from django.utils import timezone
from django.db.models import Count

Write a function get_post_details to retrieve the data for a post, this function will return a dict


( How to use type annotations in Python )

def get_post_details(post_uuid: str) -> dict : 


The dictionary returned will be containing this data: 
{
  “uuid”:”post-uuid”,
  “created_by”: {
     “name”: “User’s name”,
     “uuid”: “Users’s UUID”
   },
   “created_date”: “date”,
   “likes”: 10,
   “context”: “Post context”
}







In [15]:

def get_post_details(post_id):
    
    try:
        post = (
            Post.objects
            .select_related('user')
            .prefetch_related('likes')
            .get(
                id=post_id
            )
        )
       
    except Post.DoesNotExist:
        return {}
    
    # like_count = Likes.objects.filter(post=post, liked=True).count()
    like_count = post.likes.filter(liked=True).count()
    
    response_data = {
      "id": post.id,
      "created_by": {
         "name": post.user.name,
         "id": post.user.id
       },
       "created_date": post.created_at,
       "likes": like_count ,
       "title": post.title,
       "content": post.content
    }
    
    return response_data


    
    
    
    

In [ ]:
get_post_details(1)

{'id': 1,
 'created_by': {'name': 'PIYUSH GUPTA', 'id': 1},
 'created_date': datetime.datetime(2024, 6, 11, 11, 31, 5, 455290, tzinfo=datetime.timezone.utc),
 'likes': 1,
 'title': 'vibes of nature',
 'content': 'I am sitting on my balcony. It is spring and there is a little bit of heat in the sun. The balcony looks out over a road. The road is usually busy…'}


5. Write a function get_user_details to retrieve the data for a user, this function will return a dict. This dict should have the number of posts that the user has created and number of posts that the user has liked 


def get_user_details (user_uuid: str) -> dict : 


The dictionary returned will be containing this data: 
{
  “uuid”:”user-uuid”,
  “name”: “User’s name”
  “user-stats”: {
     “liked-posts”: 10,
     “Created-posts”: 5
  }
}






In [16]:
def get_user_details(user_id):
    try:
#         user = User.objects.get(id=user_id)
        user=User.objects.prefetch_related('posts','likes').get(id=user_id)
#         get all posts by user
        post_count=user.posts.all().count()
        liked_post_count=user.likes.filter(likes=True).count()
        
       
        
        user_details = {
            "user_id":user.id,
            "name": user.name,
            "email": user.email,
            "phone": user.phone,
            "user-stats":{
               "post_count": post_count,
               "liked_post_count": liked_post_count,
                }
        }
        
    except User.DoesNotExist:
        return {}

    
    return user_details


In [17]:
get_user_details(1)

FieldError: Cannot resolve keyword 'likes' into field. Choices are: created_at, id, liked, post, post_id, user, user_id

In [ ]:

def get_recently_liked_posts(user_id):
    try:
        user = User.objects.get(id=user_id)
        #post = Post.objects.select_related('user').prefetch_related('likes').filter()
        
    except User.DoesNotExist:
        return{}
    
    liked_posts = (
        Post.objects.filter(user=user, likes=True)
        .select_related('user')
        .prefetch_related('likes')
        .order_by('-created_at')[:5]
    )        

    
   # like_count = Likes.objects.filter(post=post, liked=True).count()
    #like_count = post.likes.filter(liked=True).count()
    
    recently_liked_posts: List[PostDetails] = []
    for post in liked_posts:
        like_count = post.likes.filter(liked=True).count()
        post_details: PostDetails = {
        
        "id": post.id,
        "title": post.title,
        "content": post.content,
        "liked_by":{
            "name": user.name,
            "id":user_id
        },
        "created_by":{
           "name": post.user.name,
            "id": post.user.id
        },
        "created_date": post.created_at,
        "likes": like_count,
        }
            
        recently_liked_posts.append(post_details)

    return recently_liked_posts



In [ ]:
get_recently_liked_posts(1)

[{'id': 1,
  'title': 'vibes of nature',
  'content': 'I am sitting on my balcony. It is spring and there is a little bit of heat in the sun. The balcony looks out over a road. The road is usually busy…',
  'liked_by': {'name': 'PIYUSH GUPTA', 'id': 1},
  'created_by': {'name': 'PIYUSH GUPTA', 'id': 1},
  'created_date': datetime.datetime(2024, 6, 11, 11, 31, 5, 455290, tzinfo=datetime.timezone.utc),
  'likes': 1}]

In [ ]:
Write a function get_popular_posts to retrieve a list of the most popular 5 posts created in the last 7 days.  

( order_by == how to sort data in SQL, order_by has to be on count of Likes on the posts)

def get_popular_posts () -> list[dict] : 


# [
#  { 
#   “uuid”:”Post-uuid”,
#   “title”: “Post title”
#   “content”: “”,
#  },
#  “created_by”: {
#      “name”: “User’s name”,
#      “uuid”: “Users’s UUID”
#    },
#    “created_date”: “date”,
#    “likes”: 20,
# { 
#   “uuid”:”Post-uuid”,
#   “title”: “Post title”
#   “content”: “”,
#   “created_by”: {
#      “name”: “User’s name”,
#      “uuid”: “Users’s UUID”
#    },
#    “created_date”: “date”,
#    “likes”: 15,
#  },
# { 
#   “uuid”:”Post-uuid”,
#   “title”: “Post title”
#   “content”: “”,
#   “created_by”: {
#      “name”: “User’s name”,
#      “uuid”: “Users’s UUID”
#    },
#    “created_date”: “date”,
#    “likes”: 11,
#  }
]






In [ ]:
from django.utils import timezone
from django.db.models import Count
from typing import List, Dict

def get_popular_posts() -> List[Dict]:
    try:
        now_time = timezone.now()
        seven_days_ago = now_time.replace(hour=0, minute=0, second=0, microsecond=0) - timezone.timedelta(days=7)
        
        popular_posts = (
            Post.objects.filter(created_at__gte=seven_days_ago)
            .annotate(like_count=Count('like'))
            .order_by('-like_count')[:5]
            .select_related('user')
        )
        
        popular_posts_data = []
        for post in popular_posts:
            popular_posts_data.append({
                "Post_details": {
                    "id": post.id,
                    "title": post.title,
                    "content": post.content,
                    "created_by": {
                        "name": post.user.name,
                        "id": post.user.id
                    },
                    "created_date": post.created_at.isoformat(),
                    "like_count": post.like_count,
                }
            })
        
        return popular_posts_data
    
    except Post.DoesNotExist:
        return []


In [ ]:
get_popular_posts()

[{'Post_details': {'id': 3,
   'title': 'Amaravati will be Andhra capital, no ‘3-capitals’',
   'content': 'A day before taking oath as Andhra Pradesh Chief Minister, Telugu Desam Party chief N Chandrababu Naidu said on Tuesday that Amaravati would be the state capital',
   'created_by': {'name': 'ayush', 'id': 2},
   'created_date': '2024-06-11T11:46:36.341939+00:00',
   'like_count': 3}},
 {'Post_details': {'id': 2,
   'title': 'A Simple Black & White Outfit',
   'content': 'don’t know about you guys, but this ‘80s baby is very happy that skorts made such a comeback in the last year. I grew up wearing skorts and loved how versatile they were, and that love hasn’t changed',
   'created_by': {'name': 'ashwin', 'id': 4},
   'created_date': '2024-06-11T11:45:31.135278+00:00',
   'like_count': 2}},
 {'Post_details': {'id': 1,
   'title': 'vibes of nature',
   'content': 'I am sitting on my balcony. It is spring and there is a little bit of heat in the sun. The balcony looks out over a roa

In [ ]:
8. Write a function get_post_type_details to retrieve a list of all the 
post_types in the DB and the post counts for all these types 

( annotate command in the ORM == Group By - Count Query in SQL )

def get_post_type_details() -> list[dict] : 


[
 { 
  “name”: “name1”
  “post-count”: 15,
 },
 { 
  “name”: “name2”
  “post-count”: 15,
 },
{ 
  “name”: “name3”
  “post-count”: 15,
 },
]







In [ ]:
def get_post_type_details():
    try:
        post_type_count=Post.objects.filter()
    
    
    